In [5]:
import math
import numpy as np
import pandas as pd

In [6]:
def get_spectrum_parameters(code: str, coef_imp: str, soil: str, zone: str) -> tuple:
    
    # accelarations
    a_gR = {'1.1': 2.5, '1.2': 2.0, '1.3': 1.5, '1.4': 1.0, '1.5': 0.6, '1.6': 0.35, 
            '2.1': 2.5, '2.2': 2.0, '2.3': 1.7, '2.4': 1.1, '2.5': 0.8,
            '0': 9.8065}
    
    index1 = ['i', 'ii', 'iii', 'iv']
    gama_f = {'CEN-1': [0.8, 1.0, 1.2, 1.4], 
            'CEN-2': [0.8, 1.0, 1.2, 1.4], 
            'PT-1': [0.65, 1.0, 1.45, 1.95], 
            'PT-2': [0.75, 1.0, 1.25, 1.5], 
            'PT-A': [0.85, 1.0, 1.15, 1.35]}
    coefs = pd.DataFrame(gama_f, index=index1)
    
    coef_imp = str.lower(coef_imp)
    a_g = a_gR[zone] * coefs.at[coef_imp, code]
    
    soil = str.upper(soil)
    index = ['A', 'B', 'C', 'D', 'E']
    if code == 'CEN-1':
        data = {'S_max': [1.0 , 1.35, 1.60, 2.0, 1.8],
                'T_B': [0.1, 0.1, 0.1, 0.1, 0.1],
                'T_C': [0.25, 0.25, 0.25, 0.3, 0.25],
                'T_D': [2.0, 2.0, 2.0, 2.0, 2.0],
                }
    elif code == 'PT-1':
        data = {'S_max': [1.0 , 1.35, 1.60, 2.0, 1.8],
                'T_B': [0.1, 0.1, 0.1, 0.1, 0.1],
                'T_C': [0.25, 0.25, 0.25, 0.3, 0.25],
                'T_D': [2.0, 2.0, 2.0, 2.0, 2.0],
                }
    elif code == 'PT-2' or code == 'PT-A':
        data = {'S_max': [1.0 , 1.35, 1.60, 2.0, 1.8],
                'T_B': [0.1, 0.1, 0.1, 0.1, 0.1],
                'T_C': [0.6, 0.6, 0.6, 0.8, 0.6],
                'T_D': [2.0, 2.0, 2.0, 2.0, 2.0],
                }
    values = pd.DataFrame(data, index=index)
    
    Smax = values.at[soil, 'S_max']

    if a_g <= 1.0:
        S = Smax 
    elif a_g >= 4.0:
        S = 1.0
    else:
        S = Smax-(Smax-1.0)*(a_g-1.0)/3.0

    TB = values.at[soil, 'T_B']
    TC = values.at[soil, 'T_C']
    TD = values.at[soil,'T_D']

    return data, S, a_g, TB, TC, TD


def get_spectrum(T: float, a_g: float, S: float, q: float, TB: float, TC: float, TD: float, beta: float=0.2) -> float:
    ag_S = a_g * S
    
    if T < TB:
        spec = ag_S * (2.0/3.0 + T / TB * (2.5 / q - 2.0/3.0))
    elif T < TC:
        spec = ag_S * 2.5 / q
    elif T < TD:
        spec = max(ag_S * 2.5 / q * (TC)/T, beta * a_g)
    else:
        spec = max(ag_S * 2.5 / q * (TC*TD)/T**2, beta * a_g)
    
    return spec

In [7]:
code = 'PT-1'
coef_imp = 'ii'
soil = 'C'
zone = '1.1'
txt = code + '_' + coef_imp + '_' + soil + '_' + zone

data, S, a_g, TB, TC, TD = get_spectrum_parameters(code, coef_imp, soil, zone)

periods = np.linspace(0.000001, TB, 15)
periods = np.append(periods, np.linspace(TB, TC, 10))
periods = np.append(periods, np.linspace(TC, TD, 10))
periods = np.append(periods, np.linspace(TD, 10, 20))

value = [get_spectrum(T, a_g, S, 2.0, TB, TC, TD, 0.2) for T in periods]
data = {'period': periods,
        'value': value}
spectrum = pd.DataFrame(data)

spectrum.to_csv('spectrum_' + txt + '.txt', index=False)